In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationEntityMemory, CombinedMemory
import os

In [36]:
from dotenv import load_dotenv
load_dotenv()

True

### Setup your environment

In [37]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") ## Put your Tavily Search API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents-memory'

## Memory types in Langchain

There are different short term memory types supported in Langchain. Important Ones are:
* Conversation Buffer Memory
* Conversation Buffer Window Memory
* Conversation Summary Memory
* Entity Memory

Read about Langchain memory types here: 

### Simple Conversation Buffer Memory

![Buffer Memory](images/buffer-memory.png)

| Pros      | Cons |
| ----------- | ----------- |
| Simple to implement      | Longer prompt -> Higher cost and Latency       |
| Passes everything to the LLM   | Very long conversations will hit the model token limit        |

In [39]:

llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [40]:
result = conversation.invoke({"human_input": "hi, I am thinking of having pasta for lunch"})
print(result['text'])

That sounds delicious! What kind of pasta are you thinking of having?


In [41]:
result = conversation.invoke({"human_input": "hi,I like white sauce. What else goes with it?"})
print(result['text'])

White sauce pasta is a classic choice! You could add some grilled chicken, shrimp, or vegetables like broccoli, mushrooms, or spinach to make it even more delicious. You could also sprinkle some parmesan cheese on top for extra flavor. Enjoy your lunch!


In [42]:
result = conversation.invoke({"human_input": "hi, add vegetarian ingredients and herbs. Also suggest good wine"})
print(result['text'])

That sounds like a great choice! For vegetarian ingredients, you can add bell peppers, zucchini, cherry tomatoes, and fresh herbs like basil or parsley. As for wine pairing, a light and crisp white wine like Pinot Grigio or Sauvignon Blanc would complement the flavors of the white sauce pasta nicely. Enjoy your meal!


In [43]:
### Look at the chat history stored
import pprint
pprint.pprint(result['chat_history'])

[HumanMessage(content='hi, I am thinking of having pasta for lunch'),
 AIMessage(content='That sounds delicious! What kind of pasta are you thinking of having?'),
 HumanMessage(content='hi,I like white sauce. What else goes with it?'),
 AIMessage(content='White sauce pasta is a classic choice! You could add some grilled chicken, shrimp, or vegetables like broccoli, mushrooms, or spinach to make it even more delicious. You could also sprinkle some parmesan cheese on top for extra flavor. Enjoy your lunch!'),
 HumanMessage(content='hi, add vegetarian ingredients and herbs. Also suggest good wine'),
 AIMessage(content='That sounds like a great choice! For vegetarian ingredients, you can add bell peppers, zucchini, cherry tomatoes, and fresh herbs like basil or parsley. As for wine pairing, a light and crisp white wine like Pinot Grigio or Sauvignon Blanc would complement the flavors of the white sauce pasta nicely. Enjoy your meal!')]


### Conversation Buffer Window Memory

![Buffer Memory](images/buffer-window-memory.png)

| Pros      | Cons |
| ----------- | ----------- |
| Simple to implement      | May not pass all the relevant context      |
| Passes only recent temporally relevant messages    |

In [44]:
llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=2, return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [45]:
result = conversation.invoke({"human_input": "hi! What are the latest trends in AI?"})
print(result['text'])

Hello! Some of the latest trends in AI include the rise of conversational AI, increased use of AI in cybersecurity, expansion of AI in healthcare, and the growth of autonomous vehicles and drones. There is also a focus on ethical AI and responsible use of AI technologies.


In [46]:
result = conversation.invoke({"human_input": "hi! Tell me about autonomous driving?"})
print(result['text'])

Autonomous driving, also known as self-driving cars or driverless cars, is a rapidly advancing technology that aims to create vehicles capable of navigating and operating without human intervention. These vehicles use a combination of sensors, cameras, radar, and artificial intelligence algorithms to perceive their surroundings, make decisions, and navigate safely on the road. Companies like Tesla, Waymo, and Uber are some of the key players in developing autonomous driving technology. The main goal of autonomous driving is to improve road safety, reduce traffic congestion, and provide more convenient transportation options.


In [47]:
result = conversation.invoke({"human_input": "Which do you think is the leader?"})
print(result['text'])

It's hard to say who the overall leader in autonomous driving technology is, as different companies have different strengths and focuses in this area. However, companies like Waymo, Tesla, and Uber are considered some of the key players in the development of autonomous driving technology. Each of these companies has made significant advancements in their autonomous driving systems and are leading the way in different aspects of the technology.


In [48]:
result = conversation.invoke({"human_input": "Is this really safe?"})
print(result['text'])

Autonomous driving technology is still in the development and testing phase, so it's important to consider that there are still risks involved. However, companies working on autonomous driving technology are continuously improving safety measures and conducting extensive testing to ensure the safety of these vehicles on the road. The goal of autonomous driving is to ultimately reduce accidents caused by human error and improve road safety overall.


In [49]:
### Look at the chat history stored
import pprint
pprint.pprint(result['chat_history'])

[HumanMessage(content='hi! Tell me about autonomous driving?'),
 AIMessage(content='Autonomous driving, also known as self-driving cars or driverless cars, is a rapidly advancing technology that aims to create vehicles capable of navigating and operating without human intervention. These vehicles use a combination of sensors, cameras, radar, and artificial intelligence algorithms to perceive their surroundings, make decisions, and navigate safely on the road. Companies like Tesla, Waymo, and Uber are some of the key players in developing autonomous driving technology. The main goal of autonomous driving is to improve road safety, reduce traffic congestion, and provide more convenient transportation options.'),
 HumanMessage(content='Which do you think is the leader?'),
 AIMessage(content="It's hard to say who the overall leader in autonomous driving technology is, as different companies have different strengths and focuses in this area. However, companies like Waymo, Tesla, and Uber ar

You can also use token buffer memory to take a fixed token limit from the past. See here for more details:
https://python.langchain.com/docs/modules/memory/types/token_buffer/

### Conversation Summary Memory

![Buffer Memory](images/summary-memory.png)

| Pros      | Cons |
| ----------- | ----------- |
| Summarizes long conversations     | Involves Time and cost to create summaries    |
| Can work with bringing in old context from long conversations    | Pertinent information many not be in summary|

In [50]:
llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationSummaryMemory(llm=ChatOpenAI(temperature=0), memory_key='chat_history', return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [51]:
result = conversation.invoke({"human_input": "hi! What are the latest trends in AI?"})
print(result['text'])

Hello! Some of the latest trends in AI include the advancement of natural language processing, the use of generative adversarial networks for creating realistic images and videos, the growth of AI in healthcare for diagnostics and personalized treatment, and the increasing integration of AI in smart home devices and autonomous vehicles. Overall, AI technology is constantly evolving and expanding into new industries and applications.


In [52]:
result = conversation.invoke({"human_input": "Tell me about autonomous driving"})
print(result['text'])

Autonomous driving, also known as self-driving or driverless cars, is an emerging technology that uses artificial intelligence and sensors to navigate vehicles without human intervention. It has the potential to revolutionize the transportation industry by improving road safety, reducing traffic congestion, and increasing efficiency. Companies like Tesla, Google's Waymo, and Uber are heavily investing in autonomous driving technology, and major car manufacturers are also developing their own self-driving systems. While there are still challenges to overcome, such as regulatory issues and public acceptance, autonomous driving is expected to become more common in the future.


In [53]:
result = conversation.invoke({"human_input": "How many years do you think till we have self driving cars?"})
print(result['text'])

It's difficult to predict an exact timeline, but many experts believe that we could see widespread adoption of self-driving cars within the next 5-10 years. The technology is rapidly advancing, and companies are making significant investments in autonomous driving systems. However, regulatory challenges, public acceptance, and technical hurdles still need to be addressed before self-driving cars become a common sight on the roads.


In [54]:
result = conversation.invoke({"human_input": "Will these cars be safe?"})
print(result['text'])

Autonomous driving technology is designed to prioritize safety, with sensors and AI algorithms constantly monitoring the vehicle's surroundings and making split-second decisions to avoid accidents. However, like any new technology, there are still challenges to overcome in ensuring the safety of self-driving cars. Companies and regulators are working on establishing standards and regulations to address safety concerns, and ongoing testing and improvements are being made to enhance the reliability and performance of autonomous driving systems. Overall, the goal is to make self-driving cars as safe, if not safer, than human-driven vehicles.


In [56]:
### Look at the chat history stored
import pprint
pprint.pprint(result['chat_history'])

[SystemMessage(content="The human asks about the latest trends in AI, and the AI responds by mentioning advancements in natural language processing, generative adversarial networks, AI in healthcare, and integration in smart home devices and autonomous vehicles. AI technology is continuously evolving and expanding into various industries and applications. The conversation shifts to autonomous driving, an emerging technology that uses AI and sensors to navigate vehicles without human intervention. Companies like Tesla, Google's Waymo, and Uber are heavily investing in autonomous driving technology, and major car manufacturers are also developing their own self-driving systems. Despite challenges, autonomous driving is expected to become more common in the future, revolutionizing the transportation industry. The human inquires about the timeline for widespread adoption of self-driving cars, and the AI explains that while it's hard to predict an exact timeframe, experts believe we could s

In [57]:
print(result)

{'human_input': 'Will these cars be safe?', 'chat_history': [SystemMessage(content="The human asks about the latest trends in AI, and the AI responds by mentioning advancements in natural language processing, generative adversarial networks, AI in healthcare, and integration in smart home devices and autonomous vehicles. AI technology is continuously evolving and expanding into various industries and applications. The conversation shifts to autonomous driving, an emerging technology that uses AI and sensors to navigate vehicles without human intervention. Companies like Tesla, Google's Waymo, and Uber are heavily investing in autonomous driving technology, and major car manufacturers are also developing their own self-driving systems. Despite challenges, autonomous driving is expected to become more common in the future, revolutionizing the transportation industry. The human inquires about the timeline for widespread adoption of self-driving cars, and the AI explains that while it's ha

### Combine 2 kinds of memory

In [58]:
llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

Summary of conversation:
{history}

Current conversation:
{recent_conversation}

Human: {input}
Chatbot:

"""

prompt = PromptTemplate(
    input_variables=["history", "recent_conversation", "input"], template=template
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
conv_memory = ConversationBufferWindowMemory(memory_key="recent_conversation", k=2, input_key='input', return_messages=True)
summary_memory = ConversationSummaryMemory(llm=ChatOpenAI(temperature=0), memory_key='history', input_key='input', return_messages=True)
memory = CombinedMemory(memories=[conv_memory, summary_memory])

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [59]:
result = conversation.invoke({"input": "help me with my Christmas shopping please"})
print(result['text'])

Of course! I'd be happy to help you with your Christmas shopping. What do you need assistance with?


In [60]:
result = conversation.invoke({"input": "for my entire family. Lets start with my parents. My dad likes to do wood working so suggest a new tool for him. My mom likes to cook so suggest a new kitchen gadget for her"})
print(result['text'])

For your dad who enjoys woodworking, I suggest a high-quality wood carving set or a new set of woodworking tools. For your mom who loves cooking, I recommend a versatile kitchen appliance like an Instant Pot or a high-end knife set. Let me know if you need more suggestions or help with anything else!


In [61]:
result = conversation.invoke({"input": "My sister likes massages and spa."})
print(result['text'])

For your sister who enjoys massages and spa treatments, you could consider getting her a gift certificate for a spa day or a set of luxurious massage oils and candles for a relaxing at-home spa experience. If you need more ideas or assistance with any other gifts, feel free to let me know!


In [62]:
result = conversation.invoke({"input": "My wife likes jewellery."})
print(result['text'])

For your wife who likes jewelry, you could consider getting her a beautiful piece of jewelry such as a necklace, bracelet, or earrings. You could also consider personalized jewelry or a jewelry box to keep her collection organized. Let me know if you need more suggestions or help with anything else!


In [63]:
pprint.pprint(result)

{'history': [SystemMessage(content='The human asks the AI to help with Christmas shopping, starting with gift ideas for their parents. The AI suggests a wood carving set or woodworking tools for the dad who enjoys woodworking, and an Instant Pot or knife set for the mom who loves cooking. The AI also recommends a spa day gift certificate or luxurious massage oils and candles for the sister who enjoys massages and spa treatments. The AI offers to provide more suggestions or assistance with anything else needed.')],
 'input': 'My wife likes jewellery.',
 'recent_conversation': [HumanMessage(content='for my entire family. Lets start with my parents. My dad likes to do wood working so suggest a new tool for him. My mom likes to cook so suggest a new kitchen gadget for her'),
                         AIMessage(content='For your dad who enjoys woodworking, I suggest a high-quality wood carving set or a new set of woodworking tools. For your mom who loves cooking, I recommend a versatile kitc

### Entity Memory

![Entity Memory](images/entity-example.png)

| Pros      | Cons |
| ----------- | ----------- |
| Organizes relevant information by entity     | Involves Time and cost to create entities    |
| More likely to capture relevant details for long conversations    | Can become very large for long conversations|

In [28]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE)

input_variables=['entities', 'history', 'input'] template='You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

In [64]:
llm = ChatOpenAI()
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationEntityMemory(llm=ChatOpenAI(temperature=0), memory_key='chat_history', return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    verbose=False,
    memory=memory
)

In [65]:
result = conversation.invoke({"input": "We are all going for a family dinner"})
result['text']

'That sounds like a wonderful way to spend time together as a family! Family dinners can be a great opportunity to bond, share stories, and enjoy delicious food. Do you have any special traditions or favorite dishes that you like to have during family dinners?'

In [66]:
result = conversation.invoke({"input": "I agree. We are going for an Italian dinner. I like to order raviola pasta. My husband like gnocci. My son Sam likes margarita pizza and my daughter Bella likes fettucini alfredo. Can you suggest starters for us?"})
result['text']

"That's wonderful to hear that you all enjoy Italian food! For starters, some popular options at Italian restaurants could include bruschetta, garlic bread, caprese salad, or antipasto platter. These dishes are typically light and flavorful, setting the tone for a delicious Italian meal. Enjoy your family dinner!"

In [67]:
result = conversation.invoke({"input": "Perfect. Me and my husband will like the Caprese salad. The kids will like to share a bruschetta. What drinks will you suggest?"})
result['text']

'For drinks to go with your Italian dinner, you could consider options like a classic Italian soda, a glass of Chianti or Sangiovese wine, or even a refreshing Limoncello cocktail. These drinks can complement the flavors of your meal and add to the overall dining experience. Enjoy your family dinner!'

In [68]:
result = conversation.invoke({"input": "Soda for the kids and prosecco for the adults"})
result['text']

"For the kids, a classic Italian soda would be a great choice, as it's a refreshing and non-alcoholic option that they may enjoy. For the adults, prosecco would be a wonderful choice to accompany the Italian meal, as it's a sparkling wine that pairs well with a variety of dishes. Enjoy your family dinner with these drink choices!"

In [69]:
conversation.memory.entity_store.store

{'Italian': 'Italian cuisine includes a variety of dishes such as ravioli pasta, gnocchi, margarita pizza, and fettucini alfredo, which are enjoyed by the family during dinner.',
 'Sam': 'Sam enjoys sharing bruschetta as a starter during family dinners.',
 'Bella': 'Bella likes to share a bruschetta.',
 'Caprese': 'Caprese is a preferred choice for the human and their husband during family dinners.',
 'Soda': 'Soda is a great non-alcoholic option for kids to enjoy during a family dinner.',
 'Prosecco': 'Prosecco is a sparkling wine that pairs well with a variety of dishes.'}

### Exercise

Explore the CONVERSATION_TOKEN_BUFFER memory from Langchain. What are its advantages/disadvantages compared to conversation window memory?

https://python.langchain.com/docs/modules/memory/types/token_buffer/